<a href="https://colab.research.google.com/github/Cody9494/LEGALSKEPSIS-DATA/blob/main/LAWSKPEPSIS_STEP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install pandas numpy tqdm faiss-cpu sentence-transformers rapidfuzz

In [2]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

import pandas as pd

path = "/content/drive/MyDrive/LAWSKEPSIS/df_master_clean.parquet"

df_master = pd.read_parquet(path)

print(df_master.shape)
df_master


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(27264, 9)


,act_id,act_type,reg_number,title,date_signed,legal_basis,recitals,articles,signatory
0,2023/207,Commission Delegated Regulation,2023/207,(EU) 2023/207 Commission Delegated Regulation ...,20221124,[Having regard to the Treaty on the Functionin...,[Annex VI to Regulation (EU) 2018/848 sets out...,[{'article_body': 'Annex VI to Regulation (EU)...,"[For the Commission ITALIC, The President ITAL..."
1,2025/1003,Commission Delegated Regulation,2025/1003,(EU) 2025/1003 Commission Delegated Regulation...,20250124,[Having regard to the Treaty on the Functionin...,[Identifying reference data should enable mark...,[{'article_body': '1. From 1 September 2026 20...,"[For the Commission ITALIC, The President ITAL..."
2,2016/182,Commission Implementing Regulation,2016/182,(EU) 2016/182 Commission Implementing Regulati...,20160211,[Having regard to the Treaty on the Functionin...,[The approval of the active substance pyrafluf...,[{'article_body': 'The approval of the active ...,"[For the Commission ITALIC, The President ITAL..."
3,2010/222,Commission Decision,2010/222,Commission Decision of 19 April 2010 20100419 ...,20100419,[Having regard to the Treaty on the Functionin...,[The European Union has stated goals in the en...,[{'article_body': 'The European Commission (he...,"[For the Commission ITALIC, Günther Oettinger ..."
4,2021/1743,Council Decision,2021/1743,(EU) 2021/1743 Council Decision of 28 Septembe...,20210928,[Having regard to the Treaty on the Functionin...,[The Treaty establishing the Transport Communi...,[{'article_body': 'The position to be taken on...,"[For the Council ITALIC, The President ITALIC,..."
...,...,...,...,...,...,...,...,...,...
27259,2008/170,Commission Decision,2008/170,Commission Decision of 27 February 2008 200802...,20080227,[Having regard to the Treaty establishing the ...,"[Under Article 75 of Regulation (EC, Euratom) ...",[{'article_body': '1. The work plan for 2008 s...,"[For the Commission ITALIC, Markos Kyprianou U..."
27260,2014/142,Council Implementing Decision,2014/142,2014/142/CFSP Council Implementing Decision of...,20140314,[Having regard to the Treaty on European Union...,"[On, the Council adopted Decision 2011/486/CFS...",[{'article_body': 'The Annex to Decision 2011/...,"[For the Council ITALIC, The President ITALIC,..."
27261,2023/2821,,,DRAFT DECISION OF THE EEA JOINT COMMITTEE No …...,20231204,[Having regard to the Agreement on the Europea...,[Regulation (EU) 2019/1149 of the European Par...,[{'article_body': 'Annex V to the EEA Agreemen...,"[For the EEA Joint Committee ITALIC, The Presi..."
27262,2023/2821,Council Decision,2023/2821,(EU) 2023/2821 Council Decision of 4 December ...,20231204,[Having regard to the Treaty on the Functionin...,[The Agreement on the European Economic Area (...,"[{'article_body': 'The position to be adopted,...","[For the Council ITALIC, The President ITALIC,..."


In [3]:
KEEP_COLS = [
    "act_id",
    "act_type",
    "reg_number",
    "title",
    "date_signed",
    "legal_basis",
    "recitals",
    "articles",
    "signatory",
]


In [4]:
for c in KEEP_COLS:
    if c not in df_master.columns:
        df_master[c] = pd.NA

df_master = df_master[KEEP_COLS].copy()
print("After column selection:", df_master.shape)


After column selection: (27264, 9)


In [5]:
cols_to_keep = [
    "act_id",
    "act_type",
    "reg_number",
    "title",
    "date_signed",
    "legal_basis",
    "recitals",
    "articles",
    "signatory"
]

In [6]:
import pandas as pd
import random
df = df_master.copy()
# Επιλογή 4 τυχαίων σειρών (με σταθερό seed για αναπαραγωγιμότητα)
sample_ids = random.sample(range(len(df)), 4)
sample = df.iloc[sample_ids]

# Ρύθμιση pandas ώστε να δείξει ΟΛΟ το περιεχόμενο
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Προβολή των 4 γραμμών πλήρως
for idx, row in sample.iterrows():
    print("="*120)
    print(f"ROW INDEX: {idx}")
    for col in cols_to_keep:
        print(f"\n{col.upper()}:\n{row[col]}")
    print("="*120 + "\n\n")

ROW INDEX: 7306

ACT_ID:
2005/38

ACT_TYPE:
Commission Decision

REG_NUMBER:
2005/38

TITLE:
Commission Decision of 27 December 2004 20041227 on the allocation of additional days absent from port to the Netherlands in accordance with Annex V to Council Regulation (EC) No 2287/2003 (notified under document number C(2004) 5269) (Only the Dutch text is authentic) (2005/38/EC)

DATE_SIGNED:
20041227

LEGAL_BASIS:
['Having regard to the Treaty establishing the European Community,'
 'Having regard to Council Regulation (EC) No 2287/2003 of fixing for 2004 the fishing opportunities and associated conditions for certain fish stocks and groups of fish stocks, applicable in Community waters and, for Community vessels, in waters where catch limitations are required, and in particular paragraph 6(c) of Annex V, 19 December 2003 20031219. Regulation as last amended by Regulation (EC) No 1928/2004 (). OJ L 344, 31.12.2003, p. 1 L 344 20031231 1 OJ L 332, 6.11.2004, p. 5 L 332 20041106 5']

RECITALS:

In [7]:
def flatten_column(col):
    return '\n'.join(col) if isinstance(col, list) else str(col)
for col in ["legal_basis", "recitals", "signatory"]:
    df_master[col] = df_master[col].apply(flatten_column)


In [8]:
df_master["embedding_text"] = (
    "TITLE: " + df_master["title"] + "\n\n" +
    "LEGAL BASIS:\n" + df_master["legal_basis"] + "\n\n" +
    "RECITALS:\n" + df_master["recitals"] + "\n\n" +
    "ARTICLES:\n" + df_master["articles"].astype(str) + "\n\n" +
    "SIGNATORY:\n" + df_master["signatory"]
)


In [9]:
df_master["token_count"] = df_master["embedding_text"].apply(lambda x: len(x.split()))
print(df_master["token_count"].describe())


count     27264.000000
mean       2075.141285
std        4767.467718
min          47.000000
25%         417.000000
50%         722.000000
75%        1526.000000
max      106952.000000
Name: token_count, dtype: float64


In [10]:
def chunk_text(text, max_tokens=1024, overlap=200):
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i+max_tokens]
        chunks.append(" ".join(chunk))
        i += max_tokens - overlap
    return chunks


In [11]:
import numpy as np
def build_embedding_text(row):
    def safe_join(value, prefix=""):
        # If it's a list or numpy array, join elements as string
        if isinstance(value, (list, np.ndarray)):
            return prefix + " ".join([str(v) for v in value if pd.notna(v)])
        # If it's a valid string
        elif isinstance(value, str):
            return prefix + value
        # If it's a scalar and not NaN
        elif pd.api.types.is_scalar(value) and pd.notna(value):
            return prefix + str(value)
        else:
            return ""

    fields = [
        safe_join(row.get("title", "")),
        safe_join(row.get("legal_basis", ""), prefix="Legal basis: "),
        safe_join(row.get("recitals", ""), prefix="Recitals: "),
        safe_join(row.get("articles", ""), prefix="Articles: "),
        safe_join(row.get("signatory", ""), prefix="Signatory: ")
    ]

    return "\n".join([f for f in fields if f.strip()])


In [12]:
df_master["embedding_text"] = df_master.apply(build_embedding_text, axis=1)


In [13]:
# Εμφάνιση 5 τυχαίων embedding_text
pd.set_option("display.max_colwidth", None)
sample = df_master[["act_id", "embedding_text"]].sample(5, random_state=42)
display(sample)


act_id  \
15245   2013/473   
21123   2017/218   
11605   2011/263   
5804    2014/922   
26719  2015/1928   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [14]:
# Έλεγχοι πληρότητας και ποιότητας
df_master["text_len"] = df_master["embedding_text"].str.len()
df_master["is_empty"] = df_master["embedding_text"].str.strip() == ""
df_master["is_nan"] = df_master["embedding_text"].isna()

print("Σύνοψη Ποιότητας:")
print(df_master[["text_len", "is_empty", "is_nan"]].describe())
print("\nΠλήθος κενού περιεχομένου:", df_master["is_empty"].sum())
print("Πλήθος NaN:", df_master["is_nan"].sum())


Σύνοψη Ποιότητας:
            text_len
count   27264.000000
mean    13462.998643
std     30665.791847
min       278.000000
25%      2759.750000
50%      4724.000000
75%     10039.250000
max    666674.000000

Πλήθος κενού περιεχομένου: 0
Πλήθος NaN: 0


In [15]:
# === Step 1: Helpers ===
def flatten_field(field_list):
    if isinstance(field_list, list):
        return " ".join(str(item).strip() for item in field_list)
    elif isinstance(field_list, str):
        return field_list.strip()
    else:
        return ""

def render_articles(articles):
    if isinstance(articles, list):
        return "\n\n".join(
            f"Article {a.get('article_number', '')}: {a.get('article_title', '')}\n{a.get('article_body', '')}"
            for a in articles if isinstance(a, dict)
        )
    elif isinstance(articles, str):
        return articles.strip()
    else:
        return ""

def build_embedding_text(row):
    parts = [
        f"Title: {flatten_field(row.get('title'))}",
        f"Legal Basis: {flatten_field(row.get('legal_basis'))}",
        f"Recitals: {flatten_field(row.get('recitals'))}",
        f"Articles:\n{render_articles(row.get('articles'))}",
        f"Signatory: {flatten_field(row.get('signatory'))}"
    ]
    return "\n\n".join(p for p in parts if p.strip())  # Skip empty


In [16]:
# Εφαρμογή με ασφάλεια
df_master["embedding_text"] = df_master.apply(build_embedding_text, axis=1)


In [17]:
import tiktoken

tokenizer = tiktoken.encoding_for_model("text-embedding-3-large")
df_master["token_len"] = df_master["embedding_text"].apply(lambda x: len(tokenizer.encode(x)))
print(df_master["token_len"].describe())


count     27264.000000
mean       1988.782864
std        5043.568186
min          21.000000
25%         506.000000
50%         826.000000
75%        1536.000000
max      131945.000000
Name: token_len, dtype: float64


In [18]:
# Έλεγξε μερικές σειρές για επιβεβαίωση
pd.set_option("display.max_colwidth", None)
display(df_master[["act_id", "embedding_text"]].sample(3, random_state=42))


act_id  \
15245  2013/473   
21123  2017/218   
11605  2011/263   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [19]:
def normalize_field(field):
    if isinstance(field, list):
        return " ".join(str(item).strip() for item in field if isinstance(item, str))
    elif isinstance(field, str):
        return field.strip()
    return ""

# Εφαρμογή για όλα τα σχετικά πεδία
for col in ["title", "legal_basis", "recitals", "articles", "signatory"]:
    df_master[col] = df_master[col].apply(normalize_field)


In [20]:
def build_embedding_text(row):
    parts = [
        f"Title: {row['title']}",
        f"Legal Basis: {row['legal_basis']}",
        f"Recitals: {row['recitals']}",
        f"Articles: {row['articles']}",
        f"Signatory: {row['signatory']}"
    ]
    return "\n\n".join(part for part in parts if part.strip())

df_master["embedding_text"] = df_master.apply(build_embedding_text, axis=1)


In [21]:
def chunk_text(text, max_tokens=1000, overlap=200):
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i + max_tokens]
        chunks.append(" ".join(chunk))
        i += max_tokens - overlap
    return chunks

df_master["embedding_chunks"] = df_master["embedding_text"].apply(chunk_text)


In [22]:
df_master["n_chunks"] = df_master["embedding_chunks"].apply(len)
print(df_master["n_chunks"].describe())


count    27264.000000
mean         2.285945
std          4.753602
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max        119.000000
Name: n_chunks, dtype: float64


In [23]:
import pandas as pd

chunk_records = []

for idx, row in df_master.iterrows():
    for i, chunk in enumerate(row["embedding_chunks"]):
        chunk_records.append({
            "act_id": row["act_id"],
            "chunk_index": i,
            "chunk_text": chunk
        })

df_chunks = pd.DataFrame(chunk_records)


In [24]:
import pandas as pd

# Έλεγχος στήλες
expected_columns = ["act_id", "chunk_index", "chunk_text"]
missing_cols = [col for col in expected_columns if col not in df_chunks.columns]
assert not missing_cols, f"Missing columns: {missing_cols}"

# Έλεγχος για κενά πεδία
print("Missing values:")
print(df_chunks.isnull().sum())

# Έλεγχος τύπων δεδομένων
print("\nData types:")
print(df_chunks.dtypes)

# Έλεγχος μήκους token ανά chunk
df_chunks["token_len"] = df_chunks["chunk_text"].apply(lambda x: len(x.split()))
print("\nToken length stats:")
print(df_chunks["token_len"].describe())

# Έλεγχος αν υπάρχουν κενά chunks ή εξαιρετικά μικρά
bad_chunks = df_chunks[df_chunks["token_len"] < 5]
print(f"\n⚠️ Chunks with <5 tokens: {len(bad_chunks)}")
if not bad_chunks.empty:
    print(bad_chunks.head(3))

# Έλεγχος για διπλότυπα
dupes = df_chunks.duplicated(subset=["act_id", "chunk_index"])
print(f"\nDuplicate chunks: {dupes.sum()}")

# Ενδεικτική προεπισκόπηση
print("\n✅ Sample preview:")
print(df_chunks.sample(3, random_state=42)[["act_id", "chunk_index", "token_len", "chunk_text"]])


Missing values:
act_id         0
chunk_index    0
chunk_text     0
dtype: int64

Data types:
act_id         object
chunk_index     int64
chunk_text     object
dtype: object

Token length stats:
count    62324.000000
mean       721.725387
std        333.433413
min          1.000000
25%        394.000000
50%       1000.000000
75%       1000.000000
max       1000.000000
Name: token_len, dtype: float64

⚠️ Chunks with <5 tokens: 76
        act_id  chunk_index                  chunk_text  token_len
334    2006/85           14           'U. Plassnik UC']          3
518   2019/618            9  of the Commission ITALIC']          4
2450  2022/155            1     the Commission ITALIC']          3

Duplicate chunks: 3221

✅ Sample preview:
          act_id  chunk_index  token_len  \
1003    2010/265            2        169   
61974   2013/186            0        700   
26834  2015/1227           27       1000   

                                                                                

In [25]:
bad_chunks

,act_id,chunk_index,chunk_text,token_len
334,2006/85,14,'U. Plassnik UC'],3
518,2019/618,9,of the Commission ITALIC'],4
2450,2022/155,1,the Commission ITALIC'],3
3830,2024/3110,16,'J. Bóka UC'],3
4372,2024/1291,1,'D. Clarinval UC'],3
4871,2023/1029,1,Leyen UC'],2
5482,2024/1288,2,von der Leyen UC'],4
5617,2015/2291,1,ITALIC'],1
9194,2008/495,2,ITALIC'],1
11244,2023/1553,1,Puchades UC'],2


In [26]:
df_chunks = df_chunks[df_chunks["token_len"] >= 5].reset_index(drop=True)


In [27]:
pd.set_option("display.max_colwidth", None)
df_chunks.sample(3, random_state=42)


act_id  chunk_index  \
253     2012/140            8   
54300   2014/622            0   
52678  2008/1099            0   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [28]:
len(df_chunks)

62248

In [29]:
# Συνδέσου με το Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Καθορισμός διαδρομής
save_path = "/content/drive/MyDrive/LAWSKEPSIS/df_chunks.parquet"

# Αποθήκευση
df_chunks.to_parquet(save_path, index=False)
print(f"Saved to: {save_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved to: /content/drive/MyDrive/LAWSKEPSIS/df_chunks.parquet
